[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/XPretrain-colab/blob/main/test.ipynb)

In [ ]:
# !wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/cut_videos.py -O /content/cut_videos.py
# !wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/download_videos.py -O /content/download_videos.py

!pip install datasets yt_dlp jsonlines

import datasets, json, jsonlines, os, subprocess
from yt_dlp import YoutubeDL

!mkdir /content/videos /content/out

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

def hhmmss(timestamp1, timestamp2):
    hh,mm,s = timestamp1.split(':')
    ss,ms = s.split('.')
    timems1 = 3600*1000*int((hh)) +  60*1000*int(mm) + 1000*int(ss) + int(ms)
    hh,mm,s = timestamp2.split(':')
    ss,ms = s.split('.')
    timems2 = 3600*1000*int((hh)) +  60*1000*int(mm) + 1000*int(ss) + int(ms)
    dur = (timems2 - timems1)/1000
    return str(dur)

def run(cmd):
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out, _ = proc.communicate()
    return out.decode('utf-8')

def extract_single_clip(sb, in_filepath, out_filepath):
    cmd = ['ffmpeg', '-ss', sb[0], '-t', hhmmss(sb[0], sb[1]),'-accurate_seek', '-i', in_filepath, '-c', 'copy',
        '-avoid_negative_ts', '1', '-reset_timestamps', '1',
        '-y', '-hide_banner', '-loglevel', 'panic', '-map', '0',out_filepath]
    run(cmd)
    if not os.path.isfile(out_filepath):
        raise Exception(f"{out_filepath}: ffmpeg clip extraction failed")

dataset = datasets.load_dataset("camenduru/microsoft-XPretrain", split='train[:1%]')
first_2_data = list(dataset)[:2]
# print(first_2_data[0])

for dump in first_2_data:
  # print(dump)
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  video_id = row['video_id']
  print(video_id)
  try:
    download_video(video_id)
    clips = row["clip"]
    for clip in clips:
      # print(clip)
      # print(clip["clip_id"], clip["span"][0], clip["span"][1])
      # print(video_id, hhmmss(clip["span"][0], clip["span"][1]))
      extract_single_clip(clip['span'], f"/content/videos/{video_id}.mp4", os.path.join("/content/out", clip['clip_id']))
  except:
    pass

In [ ]:
for dump in first_2_data:
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  clips = row["clip"]
  for clip in clips:
    name = f"/content/videos/{clip['clip_id']}"
    print(name)